In [14]:
import pandas as pd
import os
from multiprocessing import Process
import copy
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Node의 부모 Class
class Node:
    def __init__(self, node_name):
        self.node_name = node_name

# Edge의 부모 Class
class Edge:
    def __init__(self, origin, dest):
        self.origin = origin
        self.dest = dest

# Airport Class
class Airport(Node):
    def __init__(self, airport_name):
        super().__init__(airport_name)

# Flight Class
class Flight(Edge):
    def __init__(self, index, origin, dest, origin_date, dest_date, flight_duration, flight_type):
        super().__init__(origin, dest)
        self.index = index
        self.origin_date = origin_date
        self.dest_date = dest_date
        self.flight_duration = flight_duration
        self.flight_type = flight_type

    def print(self):
        print(self.index, self.origin, self.origin_date, self.dest,
              self.dest_date, self.flight_duration, self.flight_type)
        

# Stack Node의 구조
class Stackitem:
    def __init__(self, flight: Flight, flights_sequence: list = None, landing_number = None):
        if flights_sequence:
            self.flight = flight
            flights_sequence.append(flight.index)
            self.flights_sequence = flights_sequence
            self.landing_number = landing_number + 1
            # self.time = time + dt.timedelta(flight.flight_duration)
        else:
            self.flight = flight
            self.flights_sequence = [flight.index]
            self.landing_number = 1
            # self.time = flight.flight_duration;

In [15]:
# Initial Setting - Test
flight_graph = dict()
flight_excel = pd.read_excel("flight.xlsx")
flight_excel.columns = ['INDEX', 'ORIGIN', 'ORIGIN_DATE', 'DEST','DEST_DATE', 'FLIGHT_DURATION','AIRCRAFT_TYPE' ]
airports_set = set()

In [16]:
# Graph insert Node
for i in range(100) :
    airports_set.add(flight_excel.loc[i]['ORIGIN'])
    airports_set.add(flight_excel.loc[i]['DEST'])
    
for airports in airports_set :
    flight_graph[airports] = list()

# Graph insert Edge
for i in range(100) :
    flight_graph[flight_excel.loc[i]['ORIGIN']].append(Flight(flight_excel.loc[i]['INDEX'], flight_excel.loc[i]['ORIGIN'], flight_excel.loc[i]['DEST'],
                                                              flight_excel.loc[i]['ORIGIN_DATE'], flight_excel.loc[i]['DEST_DATE'],
                                                              flight_excel.loc[i]['FLIGHT_DURATION'], flight_excel.loc[i]['AIRCRAFT_TYPE']))
    

In [17]:
print(len(airports_set))

5


In [18]:
pairing_set = list()

# clustering을 통한 flights set의 크기를 줄여야지 경우의 수가 줄어듬
# 하지만 그렇다면 시간을 어떻게 배분해야하는가? 기준은? 시간 → 기종? 조금더 생각해봐야 할 거 같다.

# 여기서 Node 시작에 따라 끝나는 경우는
# 만나는 Base가 같은경우가 있고,
# 만약 끝이 Base가 아닌데 갈 곳이 없는 경우가 있다.
# 그렇다면 모든 pairing의 모든 경우를 넣어도 모든 경우가 나온다!
# 

for airport in airports_set :
    # pairing_set.append(["-" * 10, airport, "-" * 10])

    for flight in flight_graph[airport] :
        # 항공편, 항공편의 순서를 넣는다
        stack_temp = [Stackitem(flight)]
        
        while stack_temp:
            # 현재 진행 중인 항공편의 정보
            flights_information_now = stack_temp.pop()

            pairing_set.append(copy.deepcopy(flights_information_now.flights_sequence))
            # 만약 해당 항공편이 진행됨으로서 base로 간다면 해당 항공편은 더 진행할 필요가 없음
            if (flights_information_now.flight.dest == flight.origin) :
                continue
            
            else :
                pairing_set[-1].append("DeadHead")

            # Search Space Extend
            for flight_next in flight_graph[flights_information_now.flight.dest] :
                # 항공기 기종이 같은지? 시간이 가능한지? (날짜가 같다면 시간까지 비교, 다르다면 날짜 비교)
                flight_type_equal = (flights_information_now.flight.flight_type == flight_next.flight_type)
                flight_time_possible = flights_information_now.flight.dest_date < flight_next.origin_date and (flight_next.origin_date - flights_information_now.flight.dest_date).seconds/3600 >= 3
                flight_day_possible = (flight_next.dest_date  - flight.origin_date).days <= 7
                flight_landing_possible = (flights_information_now.landing_number <= 3)

                # 기종이 다르거나 시간이 불가능하면 해당 항공편은 Search에 사용하지 않음
                if (not flight_type_equal or not flight_time_possible or not flight_day_possible or not flight_landing_possible) :
                    continue

                # 기종이 같고, 시간이 가능하다면 Search를 위해 stack에 추가(항공편의 순서까지)
                temp_list = copy.deepcopy(flights_information_now.flights_sequence)
                #if (flights_information_now.flight.dest_date < flight_next.origin_date) :
                    #for i in range(0, int(flight_next.origin_date.day - flights_information_now.flight.dest_date.day)) :
                        #temp_list.append("Layover")
                        
                stack_temp.append(Stackitem(flight_next, temp_list, flights_information_now.landing_number))
                

#for pairing in pairing_set :
    #print(pairing)

# 첫번째 방안 : 모든 항공편들의 순서를 저장한다
# 그렇다면 만들 수 있는(DeadHead, Layover, Feasible) Pairing의 모든 경우가 나온다.
# 하지만 이럴때는 시간이 n^n까지 나올 수 있는 것으로 보인다.
# 여기서 경우의 수를 줄이는 조건을 넣을 때
# 첫번째 - 항공기 종류
# 두번째 - 전 항공편의 도착시간보다 현 항공편의 출발시간이 느려야한다
# 세번째 - Crew의 Trun Time이 MMT보다 크거나 같아야 한다.(Slack Time은 어떻게 될려나?)
# feasible pairing은 좀 많다. -> 그럼 DeadHead가 존재하는 pairing은 넣어야하는 것인가?
# 지연전파? 존재한다?

# stack에 넣을 때 모든 정보를 넣어버린다면? 오히려 쉬워진다. 하지만 메모리를 많이 쓰므로 최적화가 필요함(조건을 생각해야 함)

In [19]:
import re

# 엑셀 파일명
filename = 'output.xlsx'

# 데이터프레임으로 변환
df = pd.DataFrame(pairing_set)

# 엑셀로 출력
df.to_excel(filename, index=False, header=False)

In [20]:
# 비행기 기종 : {크루원 수, 비행 수당, 기본급}
salary={}

salary_excel = pd.read_excel("salary.xlsx")
for index, row in salary_excel.iterrows():
    aircraft = row['AIRCRAFT']
    crew_num = row['CREW_NUM']
    flight_salary = row['FLIGHT_SALARY']
    base_salary = row['BASE_SALARY']
    layover_cost = row['BASE_SALARY']
    salary[aircraft] = {'CREW_NUM': crew_num, 'FLIGHT_SALARY': flight_salary, 'BASE_SALARY': base_salary,'LAYOVER_COST': layover_cost}

deadhead_cost={}
salary_excel = pd.read_excel("deadhead.xlsx")
for index, row in salary_excel.iterrows():
    origin = row['ORIGIN']
    dest = row['DEST']
    cost = row['deadhead']
    deadhead_cost[origin,dest]=cost

print(deadhead_cost)

total = []
i=0

for pairing in pairing_set :
    crew_num = salary[flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['AIRCRAFT_TYPE']]['CREW_NUM']
    base_salary = salary[flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['AIRCRAFT_TYPE']]['BASE_SALARY']
    flight_salary = salary[flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['AIRCRAFT_TYPE']]['FLIGHT_SALARY']
    layover_cost =  salary[flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['AIRCRAFT_TYPE']]['LAYOVER_COST']
    layover_time = 0
    flight_time = flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['FLIGHT_DURATION']
    deadhead=0
    for idx,pair in enumerate(pairing[1:],start=1):
        if(pair != "DeadHead") :
            n = int(re.search(r'\d+', pair).group())
            flight_time+=flight_excel.loc[n-1]['FLIGHT_DURATION']
            before_flight = flight_excel.loc[int(re.search(r'\d+', pairing[idx-1]).group())-1]['ORIGIN_DATE']
            current_flight = flight_excel.loc[int(re.search(r'\d+', pair).group())-1]['ORIGIN_DATE']
            layover_time += (current_flight-before_flight).seconds/3600
        else :
            #deadhead cost 정의하기
            before_flight_airport=flight_excel.loc[int(re.search(r'\d+', pairing[idx-1]).group())-1]['DEST']
            base_airport = flight_excel.loc[int(re.search(r'\d+', pairing[0]).group())-1]['ORIGIN']
            print(before_flight_airport,base_airport)
            
            deadhead = deadhead_cost[before_flight_airport,base_airport]* crew_num
        
            
    
    print(i,pairing)
    i+=1
    print('base_salary :',base_salary,'flight_salary :', flight_salary*flight_time,'layover_cost :',0.1*layover_cost*layover_time,'deadhead :',deadhead)
    total.append(base_salary+(flight_salary*flight_time)+(0.1*layover_cost*layover_time)+deadhead)
    print('Total: ',base_salary+flight_salary*flight_time+0.1*layover_cost*layover_time)
    #print(pairing)
    #print(total)

print('Total paring :',len(pairing_set))



{('ATL', 'LGA'): 100, ('ATL', 'DTW'): 200, ('ATL', 'MSP'): 230, ('ATL', 'SLC'): 380, ('LGA', 'ATL'): 200, ('LGA', 'DTW'): 150, ('LGA', 'MSP'): 300, ('LGA', 'SLC'): 250, ('DTW', 'ATL'): 150, ('DTW', 'LGA'): 150, ('DTW', 'MSP'): 180, ('DTW', 'SLC'): 390, ('MSP', 'ATL'): 230, ('MSP', 'LGA'): 300, ('MSP', 'DTW'): 210, ('MSP', 'SLC'): 330, ('SLC', 'ATL'): 400, ('SLC', 'LGA'): 390, ('SLC', 'DTW'): 390, ('SLC', 'MSP'): 330}
ATL DTW
0 ['F7', 'DeadHead']
base_salary : 2275 flight_salary : 283.40000000000003 layover_cost : 0.0 deadhead : 1200
Total:  2558.4
ATL DTW
1 ['F11', 'DeadHead']
base_salary : 1925 flight_salary : 238.7 layover_cost : 0.0 deadhead : 1000
Total:  2163.7
SLC DTW
2 ['F12', 'DeadHead']
base_salary : 2275 flight_salary : 548.6 layover_cost : 0.0 deadhead : 2340
Total:  2823.6
ATL DTW
3 ['F14', 'DeadHead']
base_salary : 2275 flight_salary : 279.5 layover_cost : 0.0 deadhead : 1200
Total:  2554.5
SLC DTW
4 ['F14', 'F96', 'DeadHead']
base_salary : 2275 flight_salary : 962.0 layov

In [21]:
import gurobipy as gp
from gurobipy import GRB
import random

def delta(pair, num):
    for p in pair:
      if p.startswith('F'):
            if p[1:] == str(num):
               return 1
    return 0

total_flight_num = 100

#p_set = []
#cnt = 0
#pairing_set_len = len(pairing_set)
#비행편 데드헤드있는 경우만 추출 --> 100개
#for pair in pairing_set:
#   if len(pair) == 2 and pair[-1] == 'DeadHead':
#        p_set.append(pair)

#random_percent= int(0.5*len(pairing_set))

#원래 페어에서 제거       
#for pair in pairing_set :
#    pairing_set.remove(pair)



#for i in range(random_percent) :
#    random_chocie = random.choice(pairing_set)
#    p_set.append(random_chocie)
#    pairing_set.remove(random_chocie)

#print('random_percent',random_percent)

#print(p_set)
#print(len(p_set))


# 최적화 모델 생성
model = gp.Model('Crew_Pairing')


# 변수 추가
x = []

for j in range(len(pairing_set)):
    x_j = model.addVar(vtype=GRB.CONTINUOUS, name='x_%s' % j)
    x.append(x_j)
#   model.setAttr("Start", x_j, 1)
    


for j in range(total_flight_num) :
    #비행편 마다의 제약 조건 추가?..
        model.addConstr(gp.quicksum(delta(pairing_set[i],j+1)*x[i] for i in range(len(pairing_set)))==1, name='d_%s' % j)


    
#model.addConstr(gp.quicksum(x[i] for i in range(len(p_set))) <= 80, name='extra')

# objective function 설정
obj = gp.quicksum(x[i] * total[i] for i in range(len(pairing_set)))


# Set the tolerance for optimality
#model.setParam('IntFeasTol', 1e-8)

model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

if(model.Status==GRB.OPTIMAL) :
    print("최적해 찾음!!")
#print(model.ObjNVal)

# 변수 값 출력
for v in model.getVars():
    print(v.varName,v.getAttr)

#print(x,total)

# 제약 조건 슬랙 출력
for c in model.getConstrs():
   print(c.constrName,c.Slack)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 939893
Academic license - for non-commercial use only - registered to skxkswls@gmail.com
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to skxkswls@gmail.com
Optimize a model with 100 rows, 654 columns and 1676 nonzeros
Model fingerprint: 0x5cd5127e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+03, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 178 columns
Presolve time: 0.05s
Presolved: 97 rows, 476 columns, 1187 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4775008e+05   5.300000e+01   0.000000e+00      0s
      69    2.5869925e+05   0.000000e